In [1]:
# Flags MPV incidents by whether sheriffs offices or local police departments were responsible

In [2]:
# Process incident data, flag for sheriff or police department involvement

In [3]:
import pandas as pd;
import pyreadr
from matplotlib import pyplot as plt
from pandas.plotting import table
from datetime import datetime
from tabulate import tabulate

In [4]:
# load the ORI lookup table
leoka_lookup = (
    pd.read_csv(
        "../outputs/leoka_ori_type_lookup.csv",
        na_values = ["nan"],
        dtype = {"ori": str, "ori9":str},
    )
    .assign(
        ori9 = lambda x: x["ori9"].fillna(x["ori"].astype(str))
    )
)

#get unique list of sheriff ORIs
sheriff_oris = leoka_lookup[leoka_lookup["agency_type"] == "sheriffs office"]["ori9"].unique()
police_oris = leoka_lookup[leoka_lookup["agency_type"] == "local police department"]["ori9"].unique()

sheriff_oris, police_oris

(array(['AL0010000', 'AL0020000', 'AL0030000', ..., 'WY0210000',
        'WY0220000', 'WY0230000'], dtype=object),
 array(['AK0010100', 'AK0010200', 'AK0010300', ..., 'WY0230100',
        'WY0230200', 'WYDI05000'], dtype=object))

In [5]:
# read incidents from MPV dataset
incidents = (
    pd.read_excel(
        "../data/MPVDatasetDownload.xlsx",
        parse_dates = ["Date of Incident (month/day/year)"],
        sheet_name=0,
    )
    .rename(
        columns = { "ORI Agency Identifier (if available)": "ori" }
    )
    .rename_axis("incident_uid")
    .reset_index()
)

incidents.head(3)

,incident_uid,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,...,Officer Prosecuted by (Chief Prosecutor),Prosecutor Race,Prosecutor Gender,Chief Prosecutor Political Party,Chief Prosecutor Term,Officer Prosecuted by (Prosecutor in Court),Special Prosecutor?,Independent Investigation?,Prosecutor Source Link,Killed by Police 2013-24
0,0,Dario Agudelo,29,Male,Hispanic,NaN,2024-02-15,62 E Michigan Ave,Battle Creek,MI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1,Curtis Wallace,36,Male,Unknown race,NaN,2024-02-15,Adams Ave and Newtown Ave,Philadelphia,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,William Lowery,46,Male,Unknown race,NaN,2024-02-14,5th St and Val Vista St,Sheridan,WY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
# flag incidents involving sheriffs
incidents_flagged = (
    incidents
    .assign(
        # checks if the incident ori is in the list of sheriff oris
        sheriff_ori = lambda x: x["ori"].str.lower().apply( lambda ori: any( [sheriff_ori in str(ori).lower() for sheriff_ori in sheriff_oris] ) ),
        # checks if the agency name contains the word "sheriff"
        has_sheriff_in_name = lambda x: x["Agency responsible for death"].str.contains("sheriff", case=False, na=False),
        # both approaches have occasional false negatives, so use both
        sheriff = lambda x: x["sheriff_ori"] | x["has_sheriff_in_name"],
    )
    .assign(
        police_ori = lambda x: x["ori"].str.lower().apply( lambda ori: any( [police_ori in str(ori).lower() for police_ori in police_oris] ) ),
        has_police_in_name = lambda x: x["Agency responsible for death"].str.contains("police", case=False, na=False),
        police = lambda x: x["police_ori"] | x["has_police_in_name"],
        others = lambda x: ~(x["sheriff"] | x["police"]),
        
    )
)

incidents_flagged.head(3)

,incident_uid,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,...,Independent Investigation?,Prosecutor Source Link,Killed by Police 2013-24,sheriff_ori,has_sheriff_in_name,sheriff,police_ori,has_police_in_name,police,others
0,0,Dario Agudelo,29,Male,Hispanic,NaN,2024-02-15,62 E Michigan Ave,Battle Creek,MI,...,NaN,NaN,1,False,False,False,False,True,True,False
1,1,Curtis Wallace,36,Male,Unknown race,NaN,2024-02-15,Adams Ave and Newtown Ave,Philadelphia,PA,...,NaN,NaN,1,False,False,False,False,True,True,False
2,2,William Lowery,46,Male,Unknown race,NaN,2024-02-14,5th St and Val Vista St,Sheridan,WY,...,NaN,NaN,1,False,True,True,False,True,True,False


In [7]:
# MPV incidents flagged with sheriff involvement based on keyword and ORI
incidents_flagged.to_csv("../outputs/incidents_flagged.csv", index = False)

## By state

In [8]:
# shorten for easier merging
incidents_short = (
    incidents_flagged
    .assign(
        year = lambda f: f["Date of Incident (month/day/year)"].dt.year,
    )
    [["incident_uid", "State", "year", "sheriff", "police", "others"]]
)

incidents_short.head()

,incident_uid,State,year,sheriff,police,others
0,0,MI,2024,False,True,False
1,1,PA,2024,False,True,False
2,2,WY,2024,True,True,False
3,3,IL,2024,False,True,False
4,4,VA,2024,False,True,False


In [9]:
# get a table of sheriff incidents by year and state
incidents_by_state = (
    incidents_short
    .groupby(
        ["State", "year"]
    )
    .agg(
        total = ("incident_uid", "count"),
        sheriff = ("sheriff", "sum"),
        police = ("police", "sum"),
        others = ("others", "sum")
    )
    .assign(
        pct_sheriff_incidents = lambda x: x["sheriff"] / x["total"],
        pct_police_incidents = lambda x: x["police"] / x["total"],
        pct_others_incidents = lambda x: x["others"] /  x["total"],
    )
    .reset_index()
    # .set_index("State")
)

incidents_by_state.head()

,State,year,total,sheriff,police,others,pct_sheriff_incidents,pct_police_incidents,pct_others_incidents
0,AK,2013,4,0,3,1,0.0,0.750000,0.250000
1,AK,2014,2,0,1,1,0.0,0.500000,0.500000
2,AK,2015,4,0,3,1,0.0,0.750000,0.250000
3,AK,2016,7,0,5,2,0.0,0.714286,0.285714
4,AK,2017,8,0,6,2,0.0,0.750000,0.250000


In [10]:
# staff by state and year
agency_staff = pd.read_csv("../outputs/agency_staff_all_years.csv")
agency_staff.head()
    

,year,state_abb,agency_type,agencies,officers,total_staff
0,1960,AK,local police department,6,110.0,138.0
1,1960,AK,state law enforcement agency,1,0.0,0.0
2,1960,AL,local police department,106,1815.0,2023.0
3,1960,AL,sheriffs office,67,0.0,0.0
4,1960,AR,local police department,66,731.0,780.0


In [11]:
# Reshape to give staff by state and agency type for latest year available
staff_by_state = (
    agency_staff
    .loc[ lambda x: x["year"] == 2022 ]
    .groupby(
        ["agency_type", "state_abb"]
    )
    .sum()
    .reset_index()
    .pivot_table(
        index = ["state_abb"],
        columns = "agency_type",
        values = "total_staff",
        # aggfunc = "sum"
    )
    .fillna(0)
    .reset_index()
    .assign(
        # target_state = lambda x: x["state_abb"].isin(expanded_target_states),
        other = lambda x: x["state law enforcement agency"] + x["federal"] + x["special jurisdiction"],
        total_agency_staff = lambda x: x["local police department"] + x["sheriffs office"] + x["other"],
        pct_sheriff = lambda x: x["sheriffs office"] / x["total_agency_staff"],
        pct_local_police = lambda x: x["local police department"] / x["total_agency_staff"],
        pct_other = lambda x: x["other"] / x["total_agency_staff"],
    )
    [[
        "state_abb",
        "total_agency_staff",
        "local police department",
        "sheriffs office",
        "pct_sheriff",
        "pct_local_police",
        "pct_other",
    ]]
    .rename_axis(None, axis=1)
    .sort_values("pct_sheriff", ascending=False)
    .reset_index(drop=True)
)

# staffing data grouped by state. Includeds percentage of staff in sheriffs offices, and target state flag
staff_by_state.to_csv("../outputs/staff_by_state.csv", index=False)
staff_by_state.head()

,state_abb,total_agency_staff,local police department,sheriffs office,pct_sheriff,pct_local_police,pct_other
0,LA,15683.0,5717.0,9796.0,0.624625,0.364535,0.010840
1,MT,3517.0,1166.0,1959.0,0.557009,0.331533,0.111459
2,ID,5700.0,2083.0,3085.0,0.541228,0.365439,0.093333
3,FL,68042.0,24196.0,36546.0,0.537109,0.355604,0.107287
4,WY,2729.0,975.0,1350.0,0.494687,0.357274,0.148040


In [12]:
# combine incidents and staff
combined_states = (
    staff_by_state.rename( columns = {"state_abb": "State"})
    .merge(
        incidents_by_state.loc[ lambda x: x["year"] == 2022 ],
        on = "State",
        how = "left"
    )
    .rename(
        columns = {
            "total": "total_incidents",
            "sheriff": "sheriff_incidents",
            "police": "police_incidents",
            "others": "other_incidents"
        }
    )
    # .sort_values("pct_sheriff_incidents", ascending=False)
    .reset_index(drop=True)
)

combined_states.to_csv("../outputs/state_staff_and_incidents.csv", index=False)
combined_states.head()

,State,total_agency_staff,local police department,sheriffs office,pct_sheriff,pct_local_police,pct_other,year,total_incidents,sheriff_incidents,police_incidents,other_incidents,pct_sheriff_incidents,pct_police_incidents,pct_others_incidents
0,LA,15683.0,5717.0,9796.0,0.624625,0.364535,0.010840,2022,19,11,11,0,0.578947,0.578947,0.000000
1,MT,3517.0,1166.0,1959.0,0.557009,0.331533,0.111459,2022,7,2,4,1,0.285714,0.571429,0.142857
2,ID,5700.0,2083.0,3085.0,0.541228,0.365439,0.093333,2022,8,6,4,0,0.750000,0.500000,0.000000
3,FL,68042.0,24196.0,36546.0,0.537109,0.355604,0.107287,2022,70,46,27,1,0.657143,0.385714,0.014286
4,WY,2729.0,975.0,1350.0,0.494687,0.357274,0.148040,2022,10,5,5,1,0.500000,0.500000,0.100000


---
---
---